# Evaluate LLM responses with Microsoft Semantic Kernel

This notebook demonstrates how to evaluate the quality of responses generated by a large language model (LLM) using Microsoft Semantic Kernel.

**Step 1**: Install NuGet packages

To get started with Semantic Kernel, you need to install the required NuGet packages. These packages provide the core functionality for interacting with AI models and managing environment variables. Specifically:
- `Microsoft.SemanticKernel` enables you to build and run AI-powered workflows.
- `DotNetEnv` allows you to load environment variables from a `.env` file, making it easier to manage secrets and configuration settings.

In [1]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.23.0"
#r "nuget: DotNetEnv, 3.1.0"

Installed Packages DotNetEnv, 3.1.0 Microsoft.SemanticKernel, 1.23.0

**Step 2**: Read environment variables

  In this step, we load these variables from a `.env` file (if present) so that they can be accessed by the application.


In [2]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\ai-examples\03-evaluate-llm-response\..\.env


**Step 3**: Instantiate the Kernel

The Semantic Kernel is the core component that orchestrates AI services and plugins. In this step, we create and configure a Kernel instance, which will be used to interact with AI models.

In [3]:
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

//Create Kernel builder
var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = //read from environment variables
    (Environment.GetEnvironmentVariable("USE_AZURE_OPENAI") == "true",
    Environment.GetEnvironmentVariable("MODEL"),
    Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT"),
    Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY"),
    Environment.GetEnvironmentVariable("OPENAI_ORG_ID"));

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

**Step 4**: Call the Kernel

Create the prompt to evaluate the LLM response. The prompt should include instructions, the response you want to evaluate and any additional context that will help the model understand what to do.

In [8]:
var prompt = @"
You are an AI assistant. You will be given the definition of an evaluation metric for assessing the quality of an answer in a question-answering task. Your job is to compute an accurate evaluation score using the provided evaluation metric. Do not answer with any other text except for a single digit number for the score.

Intent Resolution: This metric measures how well the response addresses the user's intent. 
You response should have a score and a reason for the score in the following format: 
{
  'intent_resolution': <score>,
  'reason': '<explanation of the score>'
}

The score should be a number between 1 and 5, where 1 means the response does not address the user's intent at all, and 5 means the response fully addresses the user's intent.

Example:
user: What are the opening hours of the Eiffel Tower?
assistant: Opening hours of the Eiffel Tower are 9:00 AM to 11:00 PM.
score:{
  'intent_resolution': 5,
  'reason': 'The response directly answers the user's question with clear information.'
}

Example:
user: Plan a trip to Paris.
assistant: Paris is a beautiful city with many attractions.
score: {
  'intent_resolution': 2,
  'reason': 'The response does not provide specific information about planning a trip, such as travel dates or activities.'
}
Example:
user: Book a flight to New York.
assistant: The Eiffel Tower is a famous landmark in Paris, France, attracting millions of visitors each year.
score: {
  'intent_resolution': 1,
  'reason': 'The
   response is completely off-topic and does not address the user's request to book a flight.'
}

context: {{$Context}}
question: {{$Question}}
answer: {{$Answer}}
score:
"; 

 KernelArguments arguments = new() { { "Context",  "Paris is a beautiful city with many attractions." }, 
                                      { "Question", "What is the capital of France?" }, 
                                      { "Answer", "The capital of France is Paris." } };
var response = await kernel.InvokePromptAsync(prompt,arguments);

Console.WriteLine($"Response: {response}");

Response: {
  'intent_resolution': 5,
  'reason': 'The response directly answers the user's question with the precise fact that Paris is the capital of France.'
}
